!pip install opencv-python mediapipe sklearn matpotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [6]:
def draw_styled_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
#                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
#                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))  

In [6]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        cv2.imshow("OpenCV Feed", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] 
                     for res in results.pose_landmarks.landmark]
                   ).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] 
                   for res in results.left_hand_landmarks.landmark]
                   ).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] 
                   for res in results.right_hand_landmarks.landmark]
                   ).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     face = np.array([[res.x, res.y, res.z]
#                     for res in results.face_landmarks.landmark]
#                    ).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose, lh, rh])

In [8]:
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import TensorBoard

In [9]:
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['hello', 'thanks', 'iloveyou', 'bye', 'sorry', 'ok'])
# , 'hello', 'thanks', 'iloveyou', 'bye', 'tease', 'sorry'
no_sequences = 30
sequence_length = 30
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [20]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [22]:
print("Size of Seq and Lbl", np.array(sequences).shape, np.array(labels).shape)
X = np.array(sequences)
Y = to_categorical(labels).astype(int)
print("Size of Train and Test", X.shape, Y.shape)

Size of Seq and Lbl (180, 30, 258) (180,)
Size of Train and Test (180, 30, 258) (180, 6)


In [104]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.05)

In [11]:
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(30, 258)))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64, return_sequences=True, activation='relu')))
model.add(LSTM(32, return_sequences=False, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [112]:
# lr_callback = tensorflow.keras.callbacks.ReduceLROnPlateau(
#     monitor='loss',
#     factor=0.1,
#     patience=5,
#     min_lr=0.001
# )
# lr_callback = tensorflow.keras.callbacks.LearningRateScheduler(
# lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=5.6234e-04)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [113]:
history = model.fit(x_train, y_train, epochs=90)#, callbacks=[tb_callback, lr_callback])

Epoch 1/90
6/6 [==============================] - 4s 44ms/step - loss: 1.7961 - categorical_accuracy: 0.2924
Epoch 2/90
6/6 [==============================] - 0s 38ms/step - loss: 1.6326 - categorical_accuracy: 0.3275
Epoch 3/90
6/6 [==============================] - 0s 39ms/step - loss: 1.4121 - categorical_accuracy: 0.3801
Epoch 4/90
6/6 [==============================] - 0s 38ms/step - loss: 1.3552 - categorical_accuracy: 0.4678
Epoch 5/90
6/6 [==============================] - 0s 36ms/step - loss: 1.2174 - categorical_accuracy: 0.4971
Epoch 6/90
6/6 [==============================] - 0s 36ms/step - loss: 1.1066 - categorical_accuracy: 0.5205
Epoch 7/90
6/6 [==============================] - 0s 39ms/step - loss: 1.0373 - categorical_accuracy: 0.5322
Epoch 8/90
6/6 [==============================] - 0s 36ms/step - loss: 0.9499 - categorical_accuracy: 0.5789
Epoch 9/90
6/6 [==============================] - 0s 37ms/step - loss: 1.0150 - categorical_accuracy: 0.5731
Epoch 10/90
6/6 [==

In [12]:
# model.save('emotion.h5')

In [114]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [13]:
model = tensorflow.keras.models.load_model('emotion.h5')
yhat = model.predict(x_train)

NameError: name 'x_train' is not defined

In [117]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [118]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[142,   0],
        [  0,  29]],

       [[144,   0],
        [  0,  27]],

       [[141,   0],
        [  0,  30]],

       [[142,   0],
        [  0,  29]],

       [[142,   0],
        [  0,  29]],

       [[144,   0],
        [  0,  27]]], dtype=int64)

In [119]:
accuracy_score(ytrue, yhat)

1.0

In [21]:
sequence = []
sentence = []
threshold = 0.8

colors = [(255, 192, 192), (144, 238, 144), (135, 206, 250), (255, 192, 203), (128, 0, 128), (255, 255, 0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
#             print(actions[np.argmax(res)])
            
#             if res[np.argmax(res)] > threshold: 
#                 if len(sentence) > 0: 
#                     if actions[np.argmax(res)] != sentence[-1]:
#                         sentence.append(actions[np.argmax(res)])
#                 else:
#                     sentence.append(actions[np.argmax(res)])

#             if len(sentence) > 5: 
#                 sentence = sentence[-5:]
                
            image = prob_viz(res, actions, frame, colors)
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 29ms/step


In [15]:
cap.release()
cv2.destroyAllWindows()